In [1]:
import numpy as np
from src.NN import NN
import src.utils as utils

# Settings
csv_filename = "data/seeds_dataset.csv"
hidden_layers = [1,2] # number of nodes in hidden layers i.e. [layer1, layer2, ...]
eta = 0.1 # learning rate
n_epochs = 50 # number of training epochs
n_folds = 4 # number of folds for cross-validation
seed_crossval = 1 # seed for cross-validation
seed_weights = 1 # 

In [2]:

print("Reading '{}'...".format(csv_filename))
X, y, n_classes = utils.read_csv(csv_filename, target_name="y", normalize=True)

Reading 'data/seeds_dataset.csv'...


In [3]:
print(np.hsplit(X,7)[0])

[[ 8.76678659e-02]
 [-4.43482563e-02]
 [-2.49320656e-01]
 [-4.05655538e-01]
 [ 3.93389412e-01]
 [-2.18053680e-01]
 [-1.10356317e-01]
 [-3.53543911e-01]
 [ 2.37054530e-01]
 [-1.06882209e-01]
 [-7.94755687e-01]
 [-2.94484067e-01]
 [-3.11854609e-01]
 [ 3.03062592e-01]
 [-1.01709863e+00]
 [ 8.15153945e-04]
 [ 4.10759954e-01]
 [-6.90532433e-01]
 [-7.87807471e-01]
 [-3.11854609e-01]
 [-5.41145769e-01]
 [-6.41894914e-01]
 [ 1.67572361e-01]
 [-3.18802826e-01]
 [-3.70914453e-01]
 [ 1.47115879e-02]
 [ 3.86441195e-01]
 [ 4.14234063e-01]
 [ 7.19955609e-01]
 [-7.21411241e-02]
 [-2.52794765e-01]
 [-5.09878792e-01]
 [-5.23775226e-01]
 [-6.41894914e-01]
 [ 1.71046469e-01]
 [ 3.55562387e-02]
 [-4.19551972e-01]
 [ 1.22408951e-01]
 [-6.13306301e-03]
 [-7.56152326e-02]
 [-5.12964732e-02]
 [-2.00683138e-01]
 [ 2.68321507e-01]
 [-1.79838487e-01]
 [-2.35424223e-01]
 [-1.69416161e-01]
 [ 7.76337090e-03]
 [-1.90260812e-01]
 [-3.04518224e-02]
 [ 1.29357168e-01]
 [-1.00667630e+00]
 [-1.24638979e+00]
 [-1.3123978

In [4]:
m,n=X.shape
n

7

In [5]:
def get_splits(X):
    datasets,features=X.shape

    splitWeights=list()
    splits = np.hsplit(X,features)
    # return splits

    for o,oX in enumerate(splits):
        N, d = oX.shape

        print("Neural network model:")
        print(" input_dim = {}".format(d))
        print(" hidden_layers = {}".format(hidden_layers))
        print(" output_dim = {}".format(n_classes))
        print(" eta = {}".format(eta))
        print(" n_epochs = {}".format(n_epochs))
        print(" n_folds = {}".format(n_folds))
        print(" seed_crossval = {}".format(seed_crossval))
        print(" seed_weights = {}\n".format(seed_weights))

        # Create cross-validation folds
        idx_all = np.arange(0, N)
        idx_folds = utils.crossval_folds(N, n_folds, seed=seed_crossval) # list of list of fold indices

        # Train/evaluate the model on each fold
        acc_train, acc_valid = list(), list()
        print("Cross-validating with {} folds...".format(len(idx_folds)))
        for i, idx_valid in enumerate(idx_folds):
#  seed=seed_weights
            # Collect training and test data from folds
            idx_train = np.delete(idx_all, idx_valid)
            X_train, y_train = oX[idx_train], y[idx_train]
            X_valid, y_valid = oX[idx_valid], y[idx_valid]

            # Build neural network classifier model and train
            model = NN(input_dim=d, output_dim=n_classes,
                        hidden_layers=hidden_layers)
            model.train(X_train, y_train, eta=eta, n_epochs=n_epochs)

            # Make predictions for training and test data
            ypred_train = model.predict(X_train)
            ypred_valid = model.predict(X_valid)

            # Compute training/test accuracy score from predicted values
            acc_train.append(100*np.sum(y_train==ypred_train)/len(y_train))
            acc_valid.append(100*np.sum(y_valid==ypred_valid)/len(y_valid))

            # Print cross-validation result
            print(" Fold {}/{}: acc_train = {:.2f}%, acc_valid = {:.2f}% (n_train = {}, n_valid {})".format(i+1,n_folds, acc_train[-1], acc_valid[-1], len(X_train), len(X_valid)))

        # Print results
        print(o," : ")
        print("  -> acc_train_avg = {:.2f}%, acc_valid_avg = {:.2f}%".format(sum(acc_train)/float(len(acc_train)), sum(acc_valid)/float(len(acc_valid))))

        splitWeights.append(model.get_weights())
        
    
    return splitWeights

In [6]:
wt = get_splits(X)

Neural network model:
 input_dim = 1
 hidden_layers = [1, 2]
 output_dim = 3
 eta = 0.1
 n_epochs = 50
 n_folds = 4
 seed_crossval = 1
 seed_weights = 1

Cross-validating with 4 folds...
 Fold 1/4: acc_train = 64.44%, acc_valid = 59.09% (n_train = 135, n_valid 44)
 Fold 2/4: acc_train = 65.93%, acc_valid = 65.91% (n_train = 135, n_valid 44)
 Fold 3/4: acc_train = 31.85%, acc_valid = 25.00% (n_train = 135, n_valid 44)
 Fold 4/4: acc_train = 39.26%, acc_valid = 25.00% (n_train = 135, n_valid 44)
0  : 
  -> acc_train_avg = 50.37%, acc_valid_avg = 43.75%
node :  {'weights': [1.9820452218529776], 'output': 0.0959403929471556, 'delta': 0.005213876990259619}
node :  {'weights': [1.7839999728499583], 'output': 0.5426852602533472, 'delta': 0.03277900101672253}
node :  {'weights': [0.7176104564935332], 'output': 0.5172051617588694, 'delta': -0.018999853866824287}
node :  {'weights': [-1.0231884641706612, -0.5001446139611827], 'output': 0.30705012166336837, 'delta': 0.06414050079289797}
node :  {

In [7]:
(wt [1][1])len(new[1])

[[1.8206642662766026], [0.6886832226593076]]

In [29]:
# len(model.get_weights()[0][])
datasets,features=X.shape

In [30]:
new_nn =  NN(input_dim=features, output_dim=n_classes,
                        hidden_layers=hidden_layers,AvgWeights=new)

In [31]:
new_nn.get_weights()

node :  {'weights': [1.9820452218529776, 1.9611015001814716, 1.6701133066713387, 3.5439519566673376, 2.058783755811678, 0.6399876711795246, -2.9734380919749976], 'output': None, 'delta': None}
node :  {'weights': [[-0.01383862995798056], [1.1049675325439419]], 'output': None, 'delta': None}
node :  {'weights': [[-0.01383862995798056], [1.1049675325439419]], 'output': None, 'delta': None}
node :  {'weights': [[-0.8214073507713984, -0.8361864505068126], [-0.21618911531049692, -0.454927684769184], [-0.6565914574625031, -0.5896482058168392]], 'output': None, 'delta': None}
node :  {'weights': [[-0.8214073507713984, -0.8361864505068126], [-0.21618911531049692, -0.454927684769184], [-0.6565914574625031, -0.5896482058168392]], 'output': None, 'delta': None}
node :  {'weights': [[-0.8214073507713984, -0.8361864505068126], [-0.21618911531049692, -0.454927684769184], [-0.6565914574625031, -0.5896482058168392]], 'output': None, 'delta': None}


[[[1.9820452218529776,
   1.9611015001814716,
   1.6701133066713387,
   3.5439519566673376,
   2.058783755811678,
   0.6399876711795246,
   -2.9734380919749976]],
 [[[-0.01383862995798056], [1.1049675325439419]],
  [[-0.01383862995798056], [1.1049675325439419]]],
 [[[-0.8214073507713984, -0.8361864505068126],
   [-0.21618911531049692, -0.454927684769184],
   [-0.6565914574625031, -0.5896482058168392]],
  [[-0.8214073507713984, -0.8361864505068126],
   [-0.21618911531049692, -0.454927684769184],
   [-0.6565914574625031, -0.5896482058168392]],
  [[-0.8214073507713984, -0.8361864505068126],
   [-0.21618911531049692, -0.454927684769184],
   [-0.6565914574625031, -0.5896482058168392]]]]

In [32]:
# for oX in enumerate(splits):
N, d = X.shape

print("Neural network model:")
print(" input_dim = {}".format(d))
print(" hidden_layers = {}".format(hidden_layers))
print(" output_dim = {}".format(n_classes))
print(" eta = {}".format(eta))
print(" n_epochs = {}".format(n_epochs))
print(" n_folds = {}".format(n_folds))
print(" seed_crossval = {}".format(seed_crossval))
print(" seed_weights = {}\n".format(seed_weights))

# Create cross-validation folds
idx_all = np.arange(0, N)
idx_folds = utils.crossval_folds(N, n_folds, seed=seed_crossval) # list of list of fold indices

# Train/evaluate the model on each fold
acc_train, acc_valid = list(), list()
print("Cross-validating with {} folds...".format(len(idx_folds)))
for i, idx_valid in enumerate(idx_folds):
#  seed=seed_weights
    # Collect training and test data from folds
    idx_train = np.delete(idx_all, idx_valid)
    X_train, y_train = X[idx_train], y[idx_train]
    X_valid, y_valid = X[idx_valid], y[idx_valid]

    # Build neural network classifier model and train
    # model = NN(input_dim=d, output_dim=n_classes,
    #             hidden_layers=hidden_layers)
    new_nn.train(X_train, y_train, eta=eta, n_epochs=n_epochs)

    # Make predictions for training and test data
    ypred_train = new_nn.predict(X_train)
    ypred_valid = new_nn.predict(X_valid)

    # Compute training/test accuracy score from predicted values
    acc_train.append(100*np.sum(y_train==ypred_train)/len(y_train))
    acc_valid.append(100*np.sum(y_valid==ypred_valid)/len(y_valid))

    # Print cross-validation result
    print(" Fold {}/{}: acc_train = {:.2f}%, acc_valid = {:.2f}% (n_train = {}, n_valid {})".format(i+1,n_folds, acc_train[-1], acc_valid[-1], len(X_train), len(X_valid)))

# Print results
print(o," : ")
print("  -> acc_train_avg = {:.2f}%, acc_valid_avg = {:.2f}%".format(sum(acc_train)/float(len(acc_train)), sum(acc_valid)/float(len(acc_valid))))

Neural network model:
 input_dim = 7
 hidden_layers = [1, 2]
 output_dim = 3
 eta = 0.1
 n_epochs = 50
 n_folds = 4
 seed_crossval = 1
 seed_weights = 1

Cross-validating with 4 folds...


TypeError: can't multiply sequence by non-int of type 'float'

In [17]:
new=[]
import copy
print("wth",new)
rep = copy.deepcopy(wt)
for i,mx in enumerate(rep):
    # m2 = wt[i]
    if i==0:
        print("hitting new")
        new = mx
        print(new[1])
    else:
        for j,lx in enumerate(mx):
            if j==0 and i!=0:
                # print(new[j],"88",lx)
                
                new[j] = np.append(new[j],lx)
                print(len(new[j]),"*******************")
            else:
                new[j] = np.add(new[j],lx)
                print(len(new[j]))

for k in range(len(new)):
    if not k == 0:
        print(k)
        new[k] = np.divide(new[k],len(wt))
        # print(type(l1))
print( new )

wth []
hitting new
[[1.7839999728499583], [0.7176104564935332]]
2 *******************
2
3
3 *******************
2
3
4 *******************
2
3
5 *******************
2
3
6 *******************
2
3
7 *******************
2
3
1
2
[array([ 1.98204522,  1.9611015 ,  1.67011331,  3.54395196,  2.05878376,
        0.63998767, -2.97343809]), array([[-0.01383863],
       [ 1.10496753]]), array([[-0.82140735, -0.83618645],
       [-0.21618912, -0.45492768],
       [-0.65659146, -0.58964821]])]


In [27]:
rp=0
for fp in range(len(wt)):
    print(wt[fp][1])
    rp += wt[fp][1][0][0]
    print(rp)
print(rp/7)

[[1.7839999728499583], [0.7176104564935332]]
1.7839999728499583
[[1.8206642662766026], [0.6886832226593076]]
3.604664239126561
[[1.6531845128770661], [-0.42341311366899476]]
5.257848752003627
[[-2.529236238238605], [2.5016608956467277]]
2.7286125137650217
[[0.40174446701023525], [1.9877448607528796]]
3.130356980775257
[[-0.5913458219887041], [0.18835851088466854]]
2.5390111587865527
[[-2.6358815684924166], [2.07412789503947]]
-0.09687040970586391
-0.01383862995798056


In [ ]:
np.asarray([ 3.9273942 ,  1.9839786 ,  1.7497002 ,  3.1951442 ,  1.8707918 ,
       -0.18531519,  1.9123502 ], dtype="float32")

In [ ]:
# sl = NN
def splitLearn(X):
    m,n=X.shape
    splitWeights=list()
    splits = np.hsplit(X,n)
    # return splits

    for o,oX in enumerate(splits):